## Image processing with NumPy ##
### Files: ###
* nb101_ee_sciprog_tutorial_image_001.ipynb
* plant_4500_3000.png (file can be resized and renamed, but plotting subset of image needs to be rewritten, currently hard-coded)

All files must be in the same directory.

### Description ###
The following tutorial shows some operations that can be done in in NumPy.

To demonstrate these operations, an RGB image (1 x NxMx3 array) of a plant is used. It is split into the Red, Green and Blue bands (3 x NxM arrays). These are then used to calculate a number of vegetation indices (see below for papers).

By selecting pixels with values in certain ranges for the component bands / calculated indices, a rough segmentation of green leaves is achieved. The user is invited to experiment with the numbers used for filtering.

In [ ]:
#Import necessary packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import axes

In [ ]:
#Cancel any current customizations to the style
#plt.rcParams = plt.rcParamsDefault

#Set figure size, color scale and other style options
plt.rcParams['figure.figsize'] = [16,9]
plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['axes.grid'] = True
plt.rcParams['figure.dpi'] = 120

#print(plt.rcParams)

In [ ]:
#Open image file. Change file_name to select the desired resolution (see folder for options)
file_name = "plant_4500_3000.png"
img = mpimg.imread(file_name)
img_shape = img.shape
img_shape_mono = img.shape[:-1]

#Extract the Red, Green and Blue bands from the image
R, G, B = np.squeeze(
    np.split(
        ary = img,
        indices_or_sections = 3,
        axis = -1),
    axis = -1)

In [ ]:
#Display the initial image
plt.imshow(img)
plt.colorbar()
plt.show()

In [ ]:
#Select a subset of the image to display. Maxe sure the indices are in the range of [y,x,3],
#where x&y are the original's size in pixels and 3 is for the 3 bands (R,G,B)
img_small = img[500:1000,1500:2200,:]

plt.imshow(img_small)
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(img_small[:,:,1])
plt.colorbar()
plt.show()

In [ ]:
#Display the Red band
plt.imshow(R)
plt.colorbar()
plt.show()

In [ ]:
#Display the Green band
plt.imshow(G)
plt.colorbar()
plt.show()

In [ ]:
#Display the Blue band
plt.imshow(B)
plt.colorbar()
plt.show()

In [ ]:
#Calculate a few RGB vegetation indices.
#Based on: https://www.int-arch-photogramm-remote-sens-spatial-inf-sci.net/XLII-3/1215/2018/isprs-archives-XLII-3-1215-2018.pdf
RGBVI = (G*G-R*B)/(G*G+R*B)
GLI = (2*G - R - B) / (2*G + R + B)
VARI = (G-R) / (G+R-B)
NGRDI = (G-R) / (G+R)
TGI = G - .39*R -.61*B

In [ ]:
#Display RGBVI
plt.imshow(RGBVI)
plt.colorbar()
plt.show()

In [ ]:
#Display GLI
plt.imshow(GLI)
plt.colorbar()
plt.show()

In [ ]:
#Display VARI
plt.imshow(VARI, clim = (-1, 1))#clim included because ofoutliers located very far
plt.colorbar()
plt.show()

In [ ]:
#Display NGRDI
plt.imshow(NGRDI)
plt.colorbar()
plt.show()

In [ ]:
#Display TGI
plt.imshow(TGI)
plt.colorbar()
plt.show()

In [ ]:
#Let's "cut out" the leaves (areas where the vegetation indices are high)
#Need to define some threshold for what "high" means. Iterative process!

filter_mask = np.zeros(shape = img_shape_mono,
                       dtype = np.bool)

filter_mask[
    np.logical_and(
        np.logical_and(NGRDI > -5, B < 5),
        np.logical_and(GLI > 0.1, GLI < 0.5)
    )] = True

R_masked = np.ones_like(R)
G_masked = np.ones_like(G)
B_masked = np.ones_like(B)

R_masked[filter_mask] = R[filter_mask]
G_masked[filter_mask] = G[filter_mask]
B_masked[filter_mask] = B[filter_mask]

img_masked = np.stack([R_masked, G_masked, B_masked],
                      axis = 2)

plt.imshow(img_masked)
plt.colorbar()
plt.show()

### Attention ###

The following cells are not part of the tutorial, but were part of the process of choosing the thresholds used while filtering out the data.

In [ ]:
_ = plt.hist(R[500:2000, 2000:4000].flatten(), bins=256)

In [ ]:
_ = plt.hist(G[500:2000, 2000:4000].flatten(), bins=256)

In [ ]:
_ = plt.hist(B[500:2000, 2000:4000].flatten(), bins=256)

In [ ]:
_ = plt.hist(GLI[500:2000, 2000:4000].flatten(), bins=256, density = True)

In [ ]:
whole_image_hist, whole_edges = np.histogram(GLI.flatten(),
                                             bins = 256,
                                             density = True,
                                             range = (-1,1))
crop_image_hist, crop_edges = np.histogram(GLI[1000:1500, 3000:3500].flatten(),
                                             bins = 256,
                                             density = True,
                                             range = (-1,1))

In [ ]:
x_temp = (whole_edges[1:] + whole_edges[:-1]) / 2

#plt.gca().set_ylim(1,3.3)
plt.plot(x_temp, whole_image_hist, label = "Whole image")
plt.plot(x_temp, crop_image_hist, label = "Cropped image")
plt.plot(x_temp, crop_image_hist / whole_image_hist, label = "cropped / whole")
plt.legend()
plt.show()